#  **King County House Sales Analysis**

---

### Author: [Femi Kamau](https://www.github.com/ctrl-Karugu)

## Overview

## 1. Business Understanding

A real estate angency located in King County is looking to advice home owners about how home renovations might increase the value of their homes. The agency is looking to use the King County house data provided to determine the best renovations to make to increase the value of a home.

## 2. Data Understanding
> This phase is broken down into four tasks together with its projected outcome or output in detail:
> - Collect Initial Data
> - Describe Data
> - Explore Data
> - Verify Data Quality

There was no need to collect any data for this project as it was already provided. The data consists of house data from King County and is in .csv format.

Load Libraries

In [ ]:
# data
import numpy as np
import pandas as pd

# visualization
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import folium
import warnings

# modeling
import statsmodels.api as sm
from sklearn.metrics import mean_absolute_error
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

# statistics
import scipy.stats as stats

# styling
plt.style.use('seaborn')
sns.set_style('whitegrid')

warnings.filterwarnings('ignore')

Import Data

In [ ]:
# King County House Sales dataset is imported and assigned to the variable 'data'
data = pd.read_csv('../data/raw/kc_house_data.csv')

# The shape of the dataframe and the last 5 rows are outputted
print(data.shape) 
data.tail()

There are 21 columns and 21597 rows in the dataset:
- **Numerical Columns (15)** 
  
  `date` - Date house was sold

  `price` - Sale price (prediction target)

  `bedrooms` - Number of bedrooms

  `bathrooms` - Number of bathrooms

  `sqft_living` - Square footage of living space in the home

  `sqft_lot` - Square footage of the lot

  `floors` - Number of floors (levels) in house

  `sqft_above` - Square footage of house apart from basement

  `sqft_basement` - Square footage of the basement

  `yr_built` - Year when house was built

  `yr_renovated` - Year when house was renovated

  `lat` - Latitude coordinate

  `long` - Longitude coordinate

  `sqft_living15` - The square footage of interior housing living space for the nearest 15 neighbors
  
  `sqft_lot15` - The square footage of the land lots of the nearest 15 neighbors

- **Categorical Columns (6)**

  `id` - Unique ID for each home sold

  `waterfront` - Whether the house has a view to a waterfront

  `view` - An index from 0 to 4 of how good the view of the property was

  `condition` - An index from 1 to 5 on the condition of the house

  `grade` - An index from 1 to 13, where 1-3 falls short of building construction and design, 7 has an average level of construction and design, and 11-13 have a high quality level of construction and design

  `zipcode` - What zipcode area the house is in

In [ ]:
# Describe the data
data.info()

In [ ]:
# Visualise the missing values in the dataset
msno.bar(data, color='lightgreen', figsize=(10, 5), fontsize=8)
plt.title('Missing Values Within Dataset');

From the barplot above, we can see that the columns with missing data are `waterfront`, `view`, and `yr_renovated`. The `waterfront` column has 2376 missing values, the `view` column has 63 missing values, and the `yr_renovated` column has 3842 missing values. This accounts for 11%, 0.3%, and 18% of the total number of rows in the dataset respectively.

### 2.1 Univariate Analysis
> In this section, we'll explore each column in the dataset to see the distributions of features and obtain some useful insights. The main two parts in this section are: 
>   - Categorical Columns
>   - Numerical Columns

#### 2.1.1 Categorical Columns

There are 5 Categorical Columns in the dataset that we shall be analysing:

- `id`

- `waterfront`

- `view`

- `condition`

- `grade`

- `zipcode`

Functions to visualise the data in the categorical columns

In [ ]:
# Fuction to get the value counts of the data in the columns
def get_value_counts(df, col):
    counts = df[col].value_counts(dropna=False)
    return counts

# Function to visualise the the data in the columns
def plot_data(df, col, title):
    get_value_counts(df, col).plot(kind='bar', figsize=(10, 5), color='lightgreen', edgecolor='black')
    plt.title(title)
    plt.xticks(rotation=0);

##### 2.1.1.1 ID
> The `id` column is a unique identifier for each house sold.


The univariate analysis of the `id` column will be less about identifying the data distribution, but more about identifying the number of unique values in the column. From the count of the unique values we will be able to know if the column contains any duplicates.

In [ ]:
# Check for duplicates in the 'id' column
data.id.duplicated().sum()

We see that there are 177 duplicated ids in the dataset. This could mean that there are some houses that have been sold more than once, or it could also mean that there are some records that have been imputted in this dataset more than once. We will have to investigate this further in the data preparation phase.

##### 2.1.1.2 Waterfront
> The `waterfront` column identifies whether the house is on a waterfront or not.

In [ ]:
# Identify the unique values (and counts) in the 'waterfront' column
print(get_value_counts(data, 'waterfront'))

# Visualise the data distribution
plot_data(data, 'waterfront', 'Waterfront Column Data Distribution')    

The distribution above shows that most of the houses in the dataset are not on a waterfront. The number of houses on a waterfront is 146, which is 0.7% of the total number of houses in the dataset. The missing values in this column are 2376, which is 11% of the total number of rows in the dataset. As this is a categorical column, we will impute the missing values with the mode of the column.

##### 2.1.1.3 View
> The view column identifies the quality of view from the house.

In [ ]:
# Identify the unique values (and counts) in the 'view' column
print(get_value_counts(data, 'view'))

# Visualise the data distribution
plot_data(data, 'view', 'View Column Data Distribution')

In the distribution above, we see that majority of the houses in the dataset have a no view. Furthermore, the missing values in this columns are 63, which is 0.29% of the total number of rows in the dataset. As this is a small percentage of the total number of rows in the dataset, we can drop the rows with missing values in this column.

##### 2.1.1.4 Condition
> The condition column identifies the condition of the house.

In [ ]:
# Identify the unique values (and counts) in the 'condition' column
print(get_value_counts(data, 'condition'))

# Visualise the data distribution
plot_data(data, 'condition', 'Condition Column Data Distribution')

From the distribution above, we can see that most of the houses in the dataset are in average condition. The number of houses in average condition is 12437, this accounts for 57.6% of the total number of houses in the dataset. The number of houses in good condition are 5041, this accounts for 23.3%  of the total number of houses in the dataset. The number of houses in very good condition are 1509, this accounts for 7% of the total number of houses in the dataset. The number of houes in fair condition are 152, this accounts for 0.7% of the total number of houses in the dataset. The number of houses in poor condition are 25, this accounts for 0.1% of the total number of houses in the dataset. Furthermore, we can see that there is no missing data within this column.

##### 2.1.1.5 Grade
> The grade column identifies the quality of construction and design of the house. The grade represents the construction quality of improvements. Grades run from grade 1 to 13.

In [ ]:
# Identify the unique values (and counts) in the 'grade' column
print(get_value_counts(data, 'grade'))

# Visualise the data distribution
plot_data(data, 'grade', 'Grade Column Data Distribution')
plt.xticks(rotation=45);

From the distribution above, we see that the houses in this dataset range from grades 3-13. The column is however not evenly distributed as we can see majority of the houses with a grade of 7 (representing Average), and 8 (representing Good). Lastly, there are no missing values within this column.

##### 2.1.1.5 Zipcode
>The zipcode column identifies the zipcode area the house is in.

In [ ]:
# Identify the unique values (and counts) in the 'zipcode' column
print(get_value_counts(data, 'zipcode'))

# Visualise the data distribution
plot_data(data, 'zipcode', 'Zipcode Column Data Distribution')
plt.xticks(rotation=90);

From the distribution above, we see that the zipcode with the most houses is 98103. The zipcode with the least houses is 98039. Unlike the other categorical columns, we see more evenly distributed data in this column. There is no missing data in this column.

**Summary Of The Categorical Columns**
- The quality of the data in the categorical columns is fairly good. Other than a few missing values in the `waterfront` and `view` columns, and duplicated values in the `id` column, the data is good to work with.

#### 2.1.2 Numerical Columns

There are 15 Numerical Columns in the dataset that we shall be analysing:

- `date`

- `price`
  
- `bedrooms`
  
- `bathrooms`
  
- `sqft_living`
  
- `sqft_lot`

- `floors`
  
- `sqft_above`
  
- `sqft_basement`
  
- `yr_built`
  
- `yr_renovated`
  
- `lat`
  
- `long`
  
- `sqft_living15`
  
- `sqft_lot15`

 Functions to visualise the data in the numerical columns

In [ ]:
# Fuction that describes the statistics of the data
def describe_data(df, col):
    print(df[col].describe())

# Function to plot the histogram, kde and boxplot of the data
def plot_distribution(df, col, title, bins_=10):
    # creating a figure composed of two matplotlib.Axes objects (ax_box and ax_hist)
    f, (ax_box, ax_hist) = plt.subplots(2, sharex=True, gridspec_kw={"height_ratios": (.15, .85)}, figsize=(10, 5))
    
    # assign a graph to each ax
    sns.boxplot(df[col], ax=ax_box, color='lightgreen')
    sns.histplot(data=df, x=col, ax=ax_hist, kde=True, color='lightgreen', bins=bins_, edgecolor='black')
    plt.suptitle(title)
    plt.tight_layout();

##### 2.1.2.1 Date
> The `date` column identifies the date the house was sold.

Date can either be a categorical or numerical column. In this case, we will treat it as a numerical column.

In [ ]:
# Print all the unique values in the 'date' column
print(get_value_counts(data, 'date').index.tolist())

# Get the unique values (and counts) in the 'date' column
print(get_value_counts(data, 'date'))

From the output above, we can see that the data has been stored in string format. We will have to convert the data to datetime format in the data preparation phase. Futhermore, it seems that most of the houses were sold in either 2014 or 2015.

##### 2.1.2.2 Price
> The price column identifies the price of the house.

In [ ]:
# Describe the 'price' column
describe_data(data, 'price')

# Visualise the data distribution
plot_distribution(data, 'price', 'Price Column Data Distribution', 100)

From the distribution above, we see that the price column is skewed to the right. This means that the mean price of the homes in the dataset are . The minimum price of a house in the dataset is 78,000, and the maximum price of a house in the dataset is 7,700,000. The mean price of a house in the dataset is 540,297, and the median price of a house in the dataset is 450,000. The standard deviation of the price column is 367,368.

Looking at the kurtosis of the distribution shows that

##### 2.1.2.3 Bedrooms
> The bedrooms column identifies the number of bedrooms in the house. 

In [ ]:
# Describe the 'bedroom' column
describe_data(data, 'bedrooms')

# Visualise the data distribution
plot_distribution(data, 'bedrooms', 'Bedrooms Column Data Distribution',33)

The bedroom column distribution is not skewed as the and is normally distributed. The minimum number of bedrooms in a house in the dataset is 1, and the maximum number of bedrooms in a house in the dataset is 33. The mean number of bedrooms in a house in the dataset is 3.37, and the median number of bedrooms in a house in the dataset is 3. The standard deviation of the bedrooms column is 0.93.

##### 2.1.2.4 Bathrooms
> The bathrooms column identifies the number of bathrooms in the house.

In [ ]:
# Describe the 'bathrooms' column
describe_data(data, 'bathrooms')

# Visualise the data distribution
plot_distribution(data, 'bathrooms', 'Bathrooms Column Data Distribution', 8)

From the distribution above we can see that the bathroom column is not skewed. This is because the mean and median are almost the same. The minimum number of bathrooms in a house in the dataset is 0.5, and the maximum number of bathrooms in a house in the dataset is 8. The mean number of bathrooms in a house in the dataset is 2.12, and the median number of bathrooms in a house in the dataset is 2.25. The standard deviation of the bathrooms column is 0.77.

##### 2.1.2.5 Sqft Living
> The sqft living column identifies the square footage of the house.

In [ ]:
# Describe the 'sqft_living' column
describe_data(data, 'sqft_living')

# Visualise the data distribution
plot_distribution(data, 'sqft_living', 'Sqft Living Column Data Distribution')

From the distribution above, we can see that the sqft living column is skewed to the right. This means that the mean square footage of the homes is greater than the median. The minimum square footage of a house in the dataset is 370, and the maximum square footage of a house in the dataset is 13,540. The mean square footage of a house in the dataset is 2080, and the median square footage of a house in the dataset is 1910. The standard deviation of the sqft living column is 918.

##### 2.1.2.6 Sqft Lot
> The `sqft lot` column identifies the square footage of the lot.

In [ ]:
# Describe the 'sqft_lot' column
describe_data(data, 'sqft_lot')

# Visualise the data distribution
plot_distribution(data, 'sqft_lot', 'Sqft Lot Column Data Distribution', 100)

From the distribution above, we can see that the data is skewed to the right. This is because the mean is greater than the median. The minimum lot square footage is 520, the maximium lot square footage is 1,651,359. The mean lot square footage is 15,099, and the median lot square footage is 7618. The standard deviation of the sqft lot column is 41,419.

##### 2.1.2.7 Floors
> `floors` column identifies the number of floors in the house.

In [ ]:
# Describe the 'floors' column
describe_data(data, 'floors')

# Visualise the data distribution
plot_distribution(data, 'floors', 'Floors Column Data Distribution', 5)

From the distributions above, there is no particular trend in the floors column data. Majority of the homes in the data set have 1 floors. The minimum number of floors in a house is 1, and the maximum number of floors in a house is 3.5. The median number of floors in this dataset is 1.5, and the mean number of floors in this dataset is approximately 1.5. The standard deviation of the floors column is 0.54.

##### 2.1.2.8 Sqft Above
> The `sqft above` column identifies the square footage of the house above the ground.

In [ ]:
# Describe the 'sqft_above' column
describe_data(data, 'sqft_above')

# Visualise the data distribution
plot_distribution(data, 'sqft_above', 'Sqft Above Column Data Distribution',100)

From the distributions above, we see that the square footage above ground of the houses in this dataset is skewed to the right. This is because the mean is greater than the median. The minimum square footage above ground is 370, and the maximum square footage of a house above ground is 9,410. The mean square footage above ground is 1788, and the median square footage above ground is 1,560. The standard deviation of the sqft above column is 828.

##### 2.1.2.9 Sqft Basement
> The `sqft basement` column identifies the square footage of the basement of the house.

As this column is of the type `object`, we cannot do a distribution like the other numerical columns. Therefore, we shall be viewing the contents of the column using the same technique as the categorical columns.

In [ ]:
# Print all the unique values in the 'sqft_basement' column
print(get_value_counts(data, 'sqft_basement').index.tolist())

# Get the unique values (and counts) in the 'sqft_basement' column
print(get_value_counts(data, 'sqft_basement'))

From the output above, we can see that this is numeric data that has been converted to a string. This has probably been done because of the presence of the '?' character. In order to make this data usable, we shall be converting the the data to a a float. Furthermore, we will need to deal with the missing values ('?') in this column. The missing values account for 454 records in this dataset. This comes up to 2.1% of the total records in the dataset. As this is a fairly small percentage of the total records, we shall be dropping the records missing values.

##### 2.1.2.10 Yr Built
> The `yr built` column identifies the year the house was built.

In [ ]:
# Describe the 'yr_built' column
describe_data(data, 'yr_built')

# Visualise the data distribution
plot_distribution(data, 'yr_built', 'Year Built Column Data Distribution', 115)

From the distributions above we can see that the data is slightly skewed to the left. This is because the mean is slightly lower than the median. The oldest house in the dataset was built in 1900, and the newest house in the dataset was built in 2015. The mean year the houses in the dataset were built is 1971, and the median year the houses in the dataset were built is 1975. The standard deviation of the yr built column is 29.

##### 2.1.2.11 Yr Renovated
> The `yr renovated` column identifies the year the house was renovated.

In [ ]:
# Describe the 'yr_renovated' column
describe_data(data, 'yr_renovated')

# Visualise the data distribution
plot_distribution(data, 'yr_renovated', 'Year Renovated Column Data Distribution', 50)

From the distribution and value counts above, we can see that the data has a number of zeros. This could either be suggesting that the house has not been renovated, or that the data is missing. Furthermore, there is also some missing data in this column. We shall be analysing the data more indepth in the next phase to see how to deal with the zeros and the missing values in the column.

##### 2.1.2.12 Lat & Long
> The `lat` column identifies the latitude of the house. The `long` column identifies the longitude of the house.

In [ ]:
latlon = list(zip(data.lat, data.long))

base_map = folium.Map([data.lat.mean(), data.long.mean()], zoom_start=13)
base_map

for coord in latlon:
    folium.Marker( location=[ coord[0], coord[1]], fill_color='#43d9de', radius=8 ).add_to(base_map)
    
# export the map as HTML file
base_map.save('../images/map.html')

From the exported map above, we can see that the houses in the dataset are located in the of Washington. More importantly, we see that the houses are roughly within the same area therefore we do not need to worry about erroneous data in the lat and long columns.

##### 2.1.2.13 Sqft Living15
> The `sqft living15` square footage of interior housing living space for the nearest 15 neighbors

In [ ]:
# Describe the 'sqft_living15' column
describe_data(data, 'sqft_living15')

# Visualise the data distribution
plot_distribution(data, 'sqft_living15', 'Sqft Living15 Column Data Distribution', 100) 

From the distributions above, we can see that the data is skewed to the right. This is as a result of the mean being greater than the median. The minimum square footage of the nearest 15 neighbors is 399, and the maximum square footage of the nearest 15 neighbors is 6,210. The mean square footage of the nearest 15 neighbors is 1987, and the median square footage of the nearest 15 neighbors is 1840. The standard deviation of the sqft living15 column is 685.

##### 2.1.2.14 Sqft Lot15
> The `sqft lot15` column represents the square footage of the land lots for the nearest 15 neighbors.

In [ ]:
# Describe the 'sqft_lot15' column
describe_data(data, 'sqft_lot15')

# Visualise the data distribution
plot_distribution(data, 'sqft_lot15', 'Sqft Lot15 Column Data Distribution', 100)

In the distributions above we see a much more skewed to the right column. The minimum square footage of the nearest 15 neighbors is 651, and the maximum square footage of the nearest 15 neighbors is 871,200. The mean square footage of the nearest 15 neighbors is 12758, and the median square footage of the nearest 15 neighbors is 7620. The standard deviation of the sqft lot15 column is 27274.

**Summary Of Numerical Columns**
- The data in the numerical columns is also of fairly decent quality. Other than a few missing values in the sqft basement column and datatype corrections that need to be made to the date and sqft basement columns, the data is fairly clean. There are quite a number of outliers in the data, however, I do not think that will affect the analysis much.

## 3. Data Processing
> This phase, which is often referred to as “data munging”, prepares the final data set(s) for modeling. It has five tasks:
> - Select Data
> - Clean Data
> - Construct Data
> - Integrate Data
> - Format Data

### 3.1 Clean Data
> In this section we will be looking at the missing values in the dataset as well as the duplicate records in the dataset.

The columns that were identified to be having missing data and duplicates were:
- `id`
- `waterfront`
- `yr renovated`
- `view`


#### 3.1.1 Duplicate Records

The `id` column was identified to have duplicate records. However, we did not know if the duplicate records were actually duplicates or if they were different records with the same id. In order to find out, we shall be checking the number of unique records in the id column. 

In [ ]:
# Create a new dataframe that contains the ids that have been duplicated in the dataset.
duplicates = data[data.duplicated(['id'], keep=False)]

# Preview the duplicates dataframe
duplicates

Looking at the duplicated id records, we can see that the records are not erroneous. The ids are duplicated because the same house was sold multiple times. Therefore, we shall be keeping the records. In order to further ensure there aren't any duplicate records, we shall be checking the `date` column along with the `id` to see if the there is a house that has been sold multiple times on the same day. That would be an erroneous record. 

In [ ]:
# Check for duplicate records that have both the same id and date
duplicates[duplicates.duplicated(['id', 'date'], keep=False)]

We see that we have no records indicating that the same house has been sold multiple times on the same day. Therefore, we shall be keeping the records with duplicated ids.

#### 3.1.2 Missing Values

The columns that were identified to be having missing data were `waterfront`, `yr renovated` and `view`. We shall be dealing with the missing values in these columns. Furthermore, using the insights that were identified from the previous phase, we shall be using the type of data along with the data distribution to determine the best way to deal with the missing values.

##### 3.1.2.1 Waterfront

The `waterfront` column is a categorical column. The column has 2 unique values, 'YES' and 'NO'. With 2376 missing values, this accounted for 11% of the total records in the dataset. As this is a fairly large percentage of the total records, we shall be replacing the missing values with the mode of the column. The mode of the column is 'NO'. Therefore, we shall be replacing the missing values with 'NO'.


In [ ]:
# Fill the missing values with the mode of the column
data['waterfront'] = data['waterfront'].fillna(data['waterfront'].mode()[0])

##### 3.1.2.2 Year Renovated

The `yr renovated` column is a numerical column. With 3842 missing values, this accounted for 17.8% of the total records in the dataset. Futhermore, majority of the data in the records were zero. This could either be suggesting that the homes have never been renovated or that the data is erroneous. As there is no ideal way of daling with these values, it would be best to drop the entire column.

In [ ]:
# Drop the 'yr_renovated' column
data.drop('yr_renovated', axis=1, inplace=True)

# Preview the first five rows of the dataframe
data.head()

##### 3.1.2.3 View

The `view` column is a categorical column. With 63 missing values, this accounted for 0.3% of the total records in the dataset. As this is a small percentage of the total records, we shall be dropping the records with missing values.

In [ ]:
# Drop the missing records in the 'view' column
data['view'].dropna(inplace=True)

### 3.2 Construct Data
> In this section, we shall be deriving new attributes that will be helpful in our analysis.

We shall be creating new columns that will be useful in the analysis. The columns that we shall be looking at are the `date` and `price` columns which represents the date that the homes were sold and their price respecively. We will extract the year from the date column and creating a new column called `yr_sold`. The `yr_sold` column will be used to both check if we need to adjust the price of the homes for inflation, if so, the column will once again be used to calculate the adjusted price.

#### 3.2.1 Year Sold
> The `yr sold` column represents the year that the homes were sold.

In [ ]:
# Create a new column, 'yr_sold', from the 'date' column
data['yr_sold'] = data['date'].apply(lambda x: int(x.split('/')[-1]))

# View the values (and counts) in the 'yr_sold' column
print(get_value_counts(data, 'yr_sold'))

# Visualise the data distribution
plot_data(data, 'yr_sold', 'Year Sold Column Data Distribution')

We see that the `yr sold` column has been created and populated with the year that the homes were sold. The majority of the homes in the dataset were sold in 2014. Ultimately, the data is fairly clean and good to work with.

#### 3.2.2 Current Price
> The `current price` column represents the price of the homes adjusted for inflation.

The `current price` column could be created since there are different years involved in the sale of the homes, as a result, the prices of the homes may be different due to the different market conditions. Therefore, by creating a new column with the current price of the homes, we can fairly compare the prices of the homes in different years. However, in order to create this column, we first have to establish that the prices of the homes are indeed different due to the different market conditions. We shall be doing this by looking at the distributions of the prices of the homes sold in the different years against each other.

In [ ]:
# Create different dataframes for each year (2014 and 2015)
df_2014 = data[data['yr_sold'] == 2014]
df_2015 = data[data['yr_sold'] == 2015]


# Plot the distribution of the 'price' column for each year
fig, ax = plt.subplots(figsize=(10, 5))

plt.hist(df_2014['price'], bins=100, color='orange', alpha=0.7, label='House Prices in 2014', density=True)
plt.hist(df_2015['price'], bins=100, color='lightblue', alpha=0.7, label='House Prices in 2015', density=True)
plt.title('House Prices Over The Years')
plt.legend();

From the distribution above, we can see that there is not much difference in the prices of the homes sold in the different years. Therefore, there is no need to create a new column with the current price of the homes. We can also drop the `yr sold` column as it is no longer needed.

In [ ]:
# Drop the 'yr_sold' column
data.drop('yr_sold', axis=1, inplace=True)

### 3.3 Format Data
> In this section, we shall be re-formatting data as necessary.

The specific columns that we shall be looking at in this section are:
- `date` 
- `waterfront`
- `sqft basement`

#### 3.3.1 Date

We shall be converting the date column to a datetime object.

In [ ]:
# Convert the 'date' column to datetime format
data['date'] = pd.to_datetime(data['date'], format='%m/%d/%Y')

# Preview the first five rows of the dataframe
data.head()

#### 3.3.2 Basement Square Footage

We shall be converting the `sqft_basement` column to a numerical column. However, we first need to deal with the erroneous '?' value that we identified in the previous phase. In the previous phase we saw that it accounted for 2.1% of the total records in the dataset. As this is a fairly small percentage of the total records, we shall be dropping the records with the erroneous '?' value. This will ensure that we are not introducing any bias into the dataset. Once we have dropped the records with the erroneous '?' value, we shall be converting the remaining `sqft_basement` column values to a float.

In [ ]:
# Drop the records with a '?' in the 'sqft_basement' column
data = data[data['sqft_basement'] != '?']

# Convert the 'sqft_basement' column to float
data['sqft_basement'] = data['sqft_basement'].astype(float)

# Preview the first five rows of the dataframe
data.head()

Now that we have completely cleaned our data, we can export the cleaned data to a csv file.

In [ ]:
# Export the dataframe to a csv file
data.to_csv('../data/processed/cleaned_kc_house_data.csv', index=False)

## 4. Modeling
>  In this phase, we’ll likely build and assess various models based on several different modeling techniques. This phase has four tasks:
> - Select Modeling Techniques
> - Generate Test Design
> - Build Models
> - Assess Models

### 4.1 Select Modeling Techniques
> In this section, we shall be determining which algorithms to try

I believe that the best algorithm to try for this experiment is regression. Regression is a supervised learning algorithm that is used to predict the value of a dependent variable based on the value of the independent variables. In this case we are tying to estimate the effect that the different features of the homes has on our dependent variable, the price of the homes. As a result, we will be able to provide our stakeholder with a model that will be able to predict the most pivotal features of the homes that will have the most impact on the price of the homes.

Furthermore, as we are working with multiple features, we will be using multiple linear regression. Multiple linear regression is a regression algorithm that is used to predict the value of a dependent variable based on the value of multiple independent variables (unlike linear regression which only uses one independent variable). 

### 4.2 Build Models
> In this section, we shall be building the models.

We will first start by building a baseline model. The baseline model will be used to compare the performance of the other models that we will be building. After that, we will build our multiple linear regression model.

#### 4.2.1 Build Baseline Simple Linear Regression Model
> A baseline model is essentially a simple model that acts as a reference in a machine learning project. Its main function is to contextualize the results of trained models.

The target variable is price. Therefore, we look at the correlation coefficients for all of the predictor variables to find the one with the highest correlation with price.

In [ ]:
# Create a correlation matrix for the dataset
corr = data.corr()['price'].sort_values(ascending=False)
corr

We see that the `sqft_living` column has the highest correlation with the `price` column. This is expected since the size of the house is a major factor in determining the price of the house. We shall also create a scatter plot to view the relationship between the `sqft_living` and `price`.

In [ ]:
# Plot a scatter plot of the 'price' column against the 'sqft_living' column
plt.figure(figsize=(10, 5))
plt.scatter(data['sqft_living'], data['price'], color='lightgreen', alpha=0.7, s=10, edgecolors='black')
plt.title('Price vs Living Space')
plt.xlabel('Living Space (sqft)')
plt.ylabel('Price');

 We can now declare y and X_baseline variables, where y is a Series containing price data and X_baseline is a DataFrame containing the column with the strongest correlation (`sqft_living`).

In [ ]:
# Declare y and X_baseline variables
y = data['price']
X_baseline = data[['sqft_living']]

Next, we'll use our variables to build and fit a simple linear regression model

In [ ]:
# Create a baseline model
baseline_model = sm.OLS(y, sm.add_constant(X_baseline))
baseline_results = baseline_model.fit()

# Print the summary results of the baseline model
print(baseline_results.summary())

We can plot the regression line on top of the scatter plot earlier to see how well the model fits the data.

In [ ]:
# Plot a scatter plot of the 'price' column against the 'sqft_living' column
plt.figure(figsize=(10, 5))

# Plot the regression line of the baseline model
x = np.linspace(data.sqft_living.min(), data.sqft_living.max(), 100)
Y = baseline_results.params[0] + baseline_results.params[1] * x

plt.plot(x, Y, color='black', label='Regression Line')

plt.scatter(data['sqft_living'], data['price'], color='lightgreen', alpha=0.7, s=10, edgecolors='black')
plt.title('Price vs Living Space (Baseline Model)')
plt.xlabel('Living Space (sqft)')
plt.ylabel('Price ($)')
plt.legend();

In [ ]:
# Calculate the mean absolute error of the baseline model
baseline_mae = mean_absolute_error(y, baseline_results.predict(sm.add_constant(X_baseline)))
baseline_mae

Our most strongly correlated variable with `price` is `sqft_living`

The model is statistically significant as it explains only 50% of the variance in the data. However, it is a good starting point for our analysis. In a typical prediction, the model is off by about $173992.
  
- The intercept is about -$45130. This means that that a zero square foot house would be worth about - $45130.
  
- The coefficient of `sqft_living` is about $281. This means that for every square foot increase in the house, the price of the house increases by about $281.

#### 4.2.2 Build Iterated Multiple Linear Regression Model
> We will now iterate the baseline model by building a multiple linear regression model that will have more than one independent variable.

We will start by creating a new dataframe that will contain all of the features that we want to have in our model. After that, we will encode the categorical columns. In order to know which variables to keep in our model, we will first look at a correlation matrix. This is done in order to reduce multicollinearity. Multicollinearity is a situation in which two or more independent variables are highly correlated. This can cause problems in the model as it can lead to unstable estimates of the regression coefficients. Therefore, we will be removing the variables that are highly correlated with each other.

In [ ]:
# Declare X_iter variables
X_iter = data[['bedrooms', 'bathrooms','sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built','sqft_living15', 'sqft_lot15']]

# Preview the X_iter dataframe
X_iter

We have 4 categorical columns in our dataset. As a result, we will need to encode them in order to be able to use them in our model. We will be ordinal encoding the `condition` and `grade` columns and one-hot encoding the `waterfront` and `view` columns.

##### 4.2.2.1 Encode Categorical Columns
> We will now encode the categorical columns in the dataset.

4.2.2.1.1 Ordinal Encoding
>Ordinal encoding converts each label into integer values and the encoded data represents the sequence of labels

Using the official [King County Assessor Website](https://info.kingcounty.gov/assessor/esales/Glossary.aspx?type=r), we were able to understand that the values in the `condition` and `grade` columns are ordinal, and have been assigned a value based on the quality of the feature. Therefore, we will be ordinal encoding these columns.

In [ ]:
# Create dictionaries for mapping the ordinal numberical value
condition_dict = {'Poor': 1, 'Fair': 2, 'Average': 3, 'Good': 4, 'Very Good': 5}
grade_dict = {'3 Poor': 3, '4 Low': 4, '5 Fair': 5, '6 Low Average': 6, '7 Average': 7, '8 Good': 8, '9 Better': 9, '10 Very Good': 10, '11 Excellent': 11, '12 Luxury': 12, '13 Mansion': 13}

# Map the ordinal numerical values to the 'condition' and 'grade' columns
X_iter['condition'] = X_iter['condition'].map(condition_dict)
X_iter['grade'] = X_iter['grade'].map(grade_dict)

# Preview the dataframe
X_iter

4.2.2.1.2 One Hot Encoding
> One hot encoding is a process of converting categorical data variables so they can be provided to machine learning algorithms to improve predictions.

We shall be encoding the remaining categorical columns (`waterfront` and `view`) using one hot encoding. Furthermore, in order to avoid the "Dummy Variable Trap" (perfect multicollinearity between the independent variables), we will need to drop one of the columns created.

In [ ]:
# Encode the categorical variables
X_iter = pd.get_dummies(X_iter, columns=['waterfront', 'view'], drop_first=False)

# Preview the dataframe
X_iter

In the `waterfront` column, we shall be dropping the `waterfront_NO` column as the reference column. This will allow us to study the effect of having a house on a waterfront. In the `view` column, we shall be dropping the `view_NONE` column as the reference column. This will allow us to study the effect of having a house with a view. In addition, it is the most common value in the column.

In [ ]:
# Drop the 'waterfront_NO' and 'view_NONE' columns
X_iter. drop(['waterfront_NO', 'view_NONE'], axis=1, inplace=True)

# Preview the dataframe
X_iter

##### 4.2.2.2 Correlation Matrix
> A correlation matrix is a table showing correlation coefficients between variables

We will now analyse the correlation matrix to determine which variables to keep in our model. Along with our correlation matrix, we will also be looking at the VIF (Variance Inflation Factor) of each variable. The VIF is a measure of how much the variance of an estimated regression coefficient increases if the independent variables are correlated.

We are aiming to ensure that a correlation coefficient is less than 0.6 and a VIF is less than 5. This is because a correlation coefficient of 0.6 or higher indicates that the variables are highly correlated. A VIF of 5 or higher indicates that the variables are highly correlated.

In [ ]:
# Define function to plot the correlation matrix
def corrmatrix(df):
    plt.figure(figsize=(10, 5))

    corr = df.corr()

    # Generate a mask to only show the bottom triangle
    mask = np.triu(np.ones_like(corr, dtype=bool))

    # generate heatmap
    sns.heatmap(round(corr,2), annot=True, mask=mask, vmin=-1, vmax=1, cmap='Greens')
    plt.title('Correlation Coefficient Of Predictors')
    plt.show()
    
# Define function to print the VIF values of the predictors
def vif_df(df):
    vif_data = pd.DataFrame()
    
    # Add a constant to the dataframe
    X = df.assign(const=1)

    vif_data["feature"] = X.columns

    # calculating VIF for each feature
    vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    print(vif_data.sort_values(by='VIF', ascending=False))


# Plot the correlation matrix
corrmatrix(X_iter)

# Print the VIF values of the predictors
vif_df(X_iter)

We can see a very high VIF value for the `sqft_living`, `sqft_above`, and `sqft_basement` columns. Therefore, we will be dropping `sqft_living` from our model.

In [ ]:
# Drop the 'sqft_living' column
X_iter.drop(['sqft_living'], axis=1, inplace=True)

# Visualize the correlation matrix and the VIF dataframe
corrmatrix(X_iter)
vif_df(X_iter)

The correlation matrix shows that the `sqft_above` column still has a high correlation. Therefore, we will be dropping `sqft_above` from our model.

In [ ]:
# Drop the 'sqft_above' column
X_iter.drop(['sqft_above'], axis=1, inplace=True)

# Visualize the correlation matrix and the VIF dataframe
corrmatrix(X_iter)
vif_df(X_iter)

The correlation coefficients for the `bathrooms` column is still higher than our threshold. Therefore, we will be dropping the `bathrooms` column from our model.

In [ ]:
# Drop the 'bathrooms' column
X_iter.drop(['bathrooms'], axis=1, inplace=True)

# Visualize the correlation matrix and the VIF dataframe
corrmatrix(X_iter)
vif_df(X_iter)

Dropping the `bathrooms` column has further reduced the overall correlation in the dataset. However, the `sqft_living15` and `sqft_lot15` columns still have a high correlation. Therefore, we will be dropping them both from our model.

In [ ]:
# Drop the 'sqft_lot15' and 'sqft_living15' column
X_iter.drop(['sqft_lot15', 'sqft_living15'], axis=1, inplace=True)

# Visualize the correlation matrix and the VIF dataframe
corrmatrix(X_iter)
vif_df(X_iter)

Now that we have our VIF and correlation matrix, below the threshold, we can now build our multiple linear regression model.

##### 4.2.2.3 Build Model
> We will now build our multiple linear regression model.

In [ ]:
# Create a model
iterated_model = sm.OLS(y, sm.add_constant(X_iter))
iterated_results = iterated_model.fit()

# Print the summary results of the baseline model
print(iterated_results.summary())

Now we can compare the baseline and iterated model statistics.

In [ ]:
# Compare the baseline and iterated model mean absolute errors
iterated_mae = mean_absolute_error(y, iterated_results.predict(sm.add_constant(X_iter)))
print("Baseline Model Mean Absolute Error: ", baseline_mae)
print("Iterated Model Mean Absolute Error: ", iterated_mae)

# Compare the adjusted R-squared values of the baseline and model
print("Baseline Model Adjusted R-squared: ", baseline_results.rsquared_adj)
print("Iterated Model Adjusted R-squared: ", iterated_results.rsquared_adj)

From the model results, we can see that the model is statistically significant as it explains 60% of the variance in the data compared to the 49% in the baseline model. Furthermore, the model is off by about $147,466 compared to the $173,992 in the baseline model. This is a significant improvement. 

We will now do an analysis of the coefficients of the model.

In [ ]:
# Create a dataframe of the coefficients of the iterated model along with their p-values
results_df = pd.concat([round(iterated_results.params,3), round(iterated_results.pvalues,7)], axis=1)
results_df.columns = ["coefficient", "p-value"]
results_df

##### 4.2.2.4 Model Results Analysis

We can see that all of the variables in the model are statistically significant.

- We can see that constant value is about $5,148,727. This means that a house with no features would be worth about $5,148,727.
  
- The coefficient of `bedrooms` is $16,400 which means that for every bedroom increase in the house, the price of the house increases by about $16,400.

- The coefficient of `ssqft_lot` is $0.14 which means that for every square foot increase in the lot, the price of the house increases by about $0.14.

- The coefficient of `floors` is $78,638 which means that for every floor increase in the house, the price of the house increases by about $78,638.

- The coefficient of `condition` is $19,562 which means that for every condition rating increase in the house, the price of the house increases by about $19,562.

- The coefficient of `grade` is $205,465 which means that for every grade rating increase in the house, the price of the house increases by about $205,465.

- The coefficient of `sqft_basement` is $120 which means that for every square foot increase in the basement, the price of the house increases by about $120.

- The coefficient of `yr_built` is -$3,285 which means that for every year increase in the year the house was built, the price of the house decreases by about $3,285.

- The coefficient of `waterfront_YES` is $573,073 this means that if a house is on a waterfront, the price of the house increases by about $573,073.

- The coefficients for `view` range from $68,697 to $326,838
  
  - `view_AVERAGE` is $68,697 which means that for an average view compared to no view, the price of the house increases by about $68,697.
  
  - `view_FAIR` is $129,961 which means that for a fair view compared to no view, the price of the house increases by about $129,961.
  
  - `view_GOOD` is $134,688 which means that for a good view compared to no view, the price of the house increases by about $134,688.
  
  - `view_EXCELLENT` is $326,838 which means that for an excellent view compared to no view, the price of the house increases by about $326,838.

- This `view` outcome is surprising since we would expect that the effect of having an average view to be better than having a fair view. However, the model shows that the effect of having a fair view is better than having an average view. This could perhaps suggest that the homes in the dataset with an average view are not as good as the homes with a fair view, or that that homes with an average value have been undervalued.

## 5. Conclusion
> In this phase we will be interpreting the model results and limitations in the context of the business problem and giving reccomendations to the stakeholder based on our modeling results.

### 5.1 Recommendations

Taking this analysis back to the original business problem, the aim was to help a real estate agency advise their clients on the best possible potential renovations to make to increase the the value. After modelling the data, we can see that the best renovations are as follows:

- A view of the water is the best renovation to make. This will increase the value of the house by about $573,073. As a result this most likely means that it make the view excellent as the two features are fairly correlated. In turn, by making the view excellent, the value of the house will increase by about $326,838.

- The second best renovation to make is to improve the grade of the house. This will increase the value of the house by about $205,465 for every grade. A house with the maximum grade of 13 will be worth about $2,700,000.

- The third best renovation to make is to increase the number of floors in the house. This will increase the value of the house by about $78,638 for every floor. However, it is worth mentioning that our data only had 3.5 floors as the maximum. Therefore, it is unlikely that this statistic would apply to a house with more than 3.5 floors.

- Increasing the number of bedrooms in the house will increase the value of the house by about $16,400 for every bedroom. However, it is worth mentioning that our data only had 10 bedrooms as the maximum. Therefore, it is unlikely that this statistic would apply to a house with more than 10 bedrooms.


### 5.2 Limitations

Though our model did show a significant increase in the accuracy of the model, there are still some limitations to the model. These limitations are as follows:

- The data in the dataset is from 2014 and 2015. Therefore, it may not be able to account for changes in the housing market since then. As a result the model may not be able to predict the value of a house in 2022. 

- In order to improve the value of a house, we would need to understand the market (i.e. what buyers are looking for). Therefore, by not having this information, we are unable to advise our clients on the best renovations to make. It is possible to build the most expensive house in the world, but if it is not what buyers are looking for, then it will not be sold. There is no value in that.

- By using a correlation threshold of 0.6, we may have ignored dropping some freatures with a high correlation. This may have led to multicollinearity in the model. As a result, the model may not be able to predict the value of a house accurately.